In [1]:
sc

In [2]:
# imports
import pandas as pd
import numpy as np
from geopy.distance import great_circle
import csv

In [6]:
taxi_aug14 = sc.textFile('taxi_aug14_1000.csv'   , use_unicode=False).cache()
uber_aug14 = sc.textFile('uber-raw-data-aug14.csv', use_unicode=False).cache()
# list(enumerate(taxi_aug14.first().split(',')))
# list(enumerate(uber_aug14.first().split(',')))

In [4]:
queensCenter = ((40.800760+40.542920)/2,(-73.700272-73.962616)/2)
brookCenter = ((40.739877+40.57042)/2,(-73.864754-74.04344)/2)
bronxCenter = ((40.915255+40.785743)/2,(-73.765274-73.933406)/2)
manhattanCenter = ((40.874663+40.701293)/2,(-73.910759-74.018721)/2)
siCenter = ((40.651812+40.477399)/2,(-74.034547-74.259090)/2)

boroughDict = {}
boroughDict["queens"] = queensCenter
boroughDict["brooklyn"] = brookCenter
boroughDict["bronx"] = bronxCenter
boroughDict["manhattan"] = manhattanCenter
boroughDict["staten"] = siCenter

#This function returns the string representation of the likely borough, given a set of latitude/longitude coordinates
#If the distance to the borough center is too far away from the closest borough, we assume that the location
#is outside of NYC
def get_closest_borough(latitude,longitude,max_dist = 20):
    global boroughDict
    borough_distances = {borough:great_circle(boroughDict[borough],(latitude,longitude)).miles for borough in boroughDict}
    min_borough = min(borough_distances, key=borough_distances.get)
    if borough_distances[min_borough] < max_dist:
        return min_borough 
    else:
        return "outside_nyc"

In [10]:
def extractTaxi(partId, records):
    '''if partId==0:
        records.next()'''
    reader = csv.reader(records)
    for row in reader:
        (pickup, lon, lat, boro) = (row[1], row[5], row[6], get_closest_borough(row[6],row[5]))
        yield pickup, lon, lat, boro

trdd = taxi_aug14.mapPartitionsWithIndex(extractTaxi)
trdd.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\python\pyspark\rdd.py", line 1338, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-10-971509c2956c>", line 6, in extractTaxi
NameError: global name 'get_closet_borough' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.Pyt